# MovieLens-20m 기준 Recall@10 계산
대회에서 사용되는 원본 데이터 : MovieLens-20m

In [2]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [3]:
# 학습에 사용된 user만 uniq_user에 저장
train_ratings_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')
uniq_user = train_ratings_df['user'].unique()
print (f"Number of users : {len(uniq_user)}")

Number of users : 31360


In [4]:
uim = pd.read_csv("/opt/ml/input/workspace/CF/Non-DL/FISM_ver2/user_movie_interaction.csv")

In [5]:
uim_np = uim.to_numpy()[:, 1:]

In [6]:
uim_np

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [7]:
len(uim_np[0].nonzero()[0])

376

In [8]:
# json 으로 부터 정답 파일 불러오기
with open("answers.json", "r") as json_file:
    answer = json.load(json_file)
    
# answer example
# "11" : [1093, 1077, 3396, 671, 1982, 3991, 43936, ...]
# "14" : [...]
# movielens 원본 데이터에는 시청했지만, 학습데이터에 없는 영화들을 Dict로 저장

### submission 경로 수정해서 실행
default : `./submission.csv`

In [20]:
# movielens-20m과 submission을 비교하여 Recall@10 값 계산
submission_ub = pd.read_csv('/opt/ml/input/workspace/TestRecall/output/UB_submission.csv') # TODO: submission_file path에 맞게 수정!
submission_mvae = pd.read_csv('/opt/ml/input/workspace/CF/DL CF/Mult-VAE/submission_MVAE_200_rm.csv')
submission_dae = pd.read_csv('/opt/ml/input/workspace/CF/DL CF/Mult-VAE/submission_DAE_150.csv')
submission_b4r = pd.read_csv('/opt/ml/input/workspace/BERT4Rec/output/submission_B4R_9_T10PF.csv')
recall_result = []
cnt = 0
hit_length = list()
bad_users = list()
good_users = list()
bad_num = 0.3
good_num = 0.5
# 각 유저마다 recall@10 계산하여 list에 저장
for user in tqdm(uniq_user):
    submission_by_user_ub = submission_ub[submission_ub['user'] == user]['item']
    submission_by_user_mvae = submission_mvae[submission_mvae['user'] == user]['item']
    submission_by_user_b4r = submission_b4r[submission_b4r['user'] == user]['item']
    submission_by_user_dae = submission_dae[submission_dae['user'] == user]['item']

    hit_ub = 0
    hit_mvae = 0
    hit_dae = 0
    hit_b4r = 0

    for item in submission_by_user_ub:
        if item in answer[str(user)]:
            hit_ub += 1
    
    for item in submission_by_user_mvae:
        if item in answer[str(user)]:
            hit_mvae += 1
    
    for item in submission_by_user_dae:
        if item in answer[str(user)]:
            hit_dae += 1
    
    for item in submission_by_user_b4r:
        if item in answer[str(user)]:
            hit_b4r += 1
    
    if hit_b4r/10 < bad_num and hit_dae/10 < bad_num and hit_mvae/10 < bad_num and hit_ub/10 < bad_num :
        bad_users.append((user, cnt, len(uim_np[cnt].nonzero()[0])))
    
    if hit_ub/10 > good_num and hit_b4r/10 > good_num and hit_dae/10 > good_num and hit_mvae/10 > good_num :
        good_users.append((user, cnt, len(uim_np[cnt].nonzero()[0])))

    recall_result.append(hit_ub / 10)
    #print(hit/10, len(uim_np[cnt].nonzero()[0]))
    
    #hit_length.append((hit/10, len(uim_np[cnt].nonzero()[0])))
    hit_length.append((hit_ub/10, hit_mvae/10, hit_dae/10, hit_b4r/10))

    cnt +=1
# 전체 유저의 Recall@10의 평균 출력
print (f"Predicted submission result of Recall@10 = {np.average(recall_result)}")


100%|██████████| 31360/31360 [01:39<00:00, 314.89it/s]

Predicted submission result of Recall@10 = 0.32253507653061225


In [21]:
print(len(bad_users)) # bad_num = 0.2 -> 5741명
print(len(good_users)) # good_num = 0.5 -> 1777명

10779
1777


In [22]:
bad_users_np = np.array(bad_users)
good_users_np = np.array(good_users)

In [23]:
print(bad_users_np.mean(axis=0)) # 평균 시청 개수 108개
print(good_users_np.mean(axis=0)) # 평균 시청 개수 168개

[68840.77270619 15673.114018     114.88579646]
[68093.92628025 15503.88688801   168.92290377]


In [90]:
# 예측 못하는 유저들은 어떤 특성을 갖고 있을까?
# 1. 총 영화 시청 수
# 2. 시청한 영화의 시점, 시청한 기간
# 3. 시청한 영화의 장르 -> 편차가 심한가? = 데이터에 Noise가 많은가?
# 4. 


In [24]:
bad_user_id = bad_users_np[:,0]
bad_user_index = bad_users_np[:,1]

In [25]:
pd.DataFrame(bad_user_id).to_csv("bad_user_id03.csv", index=False, header=False)
pd.DataFrame(bad_user_index).to_csv("bad_user_index03.csv", index=False, header=False)

In [26]:
bad1 = pd.read_csv("bad_user_id03.csv", header=None).to_numpy().reshape(-1)

In [27]:
bad2 = pd.read_csv("eda_bad_user.csv", header=None).to_numpy().reshape(-1)

In [28]:
counter = 0
for user in bad1 :
    if user in bad2 :
        counter +=1

In [29]:
counter

4531